In [ ]:
import math
import numpy as np

def _constants():
    # Physical constants
    e = 1.602176634e-19          # Coulomb
    hbar = 1.054571817e-34       # J*s
    kB_eV = 8.617333262145e-5    # eV/K
    eV_to_J = 1.602176634e-19    # J/eV
    return e, hbar, kB_eV, eV_to_J

def _grid_ok(x, delta=0.1, tol=1e-9):
    return abs(x/delta - round(x/delta)) <= tol

def _build_lattice():
    # 5x4x2 periodic in x,y; 2 layers in z
    Lx, Ly, Lz = 5, 4, 2
    def idx(x,y,z):
        return (z*Ly + y)*Lx + x
    N = Lx*Ly*Lz

    # Undirected edges labeled by orientation
    edges = []  # (u,v,kind) where kind in {"x","y","z"}
    for z in range(Lz):
        for y in range(Ly):
            for x in range(Lx):
                u = idx(x,y,z)
                # +x edge
                v = idx((x+1)%Lx, y, z)
                edges.append((u,v,"x"))
                # +y edge
                v = idx(x, (y+1)%Ly, z)
                edges.append((u,v,"y"))
                # +z edge (only between layers; no periodic in z)
                if z+1 < Lz:
                    v = idx(x,y,z+1)
                    edges.append((u,v,"z"))

    # Electrodes
    S = [idx(0,y,z) for z in range(Lz) for y in range(Ly)]
    T = [idx(2,y,z) for z in range(Lz) for y in range(Ly)]
    return N, edges, S, T

class _Dinic:
    def __init__(self, n):
        self.n = n
        self.adj = [[] for _ in range(n)]
    def add_edge(self, u, v, cap):
        # forward
        self.adj[u].append([v, cap, len(self.adj[v])])
        # backward
        self.adj[v].append([u, 0.0, len(self.adj[u])-1])
    def max_flow(self, s, t):
        flow = 0.0
        INF = 1e100
        while True:
            level = [-1]*self.n
            q = [s]
            level[s] = 0
            for u in q:
                for v, cap, rev in self.adj[u]:
                    if cap > 1e-15 and level[v] < 0:
                        level[v] = level[u] + 1
                        q.append(v)
            if level[t] < 0:
                break
            it = [0]*self.n
            def dfs(u, f):
                if u == t:
                    return f
                for i in range(it[u], len(self.adj[u])):
                    it[u] = i
                    v, cap, rev = self.adj[u][i]
                    if cap > 1e-15 and level[v] == level[u] + 1:
                        ret = dfs(v, min(f, cap))
                        if ret > 0:
                            # push
                            self.adj[u][i][1] -= ret
                            self.adj[v][rev][1] += ret
                            return ret
                return 0.0
            while True:
                pushed = dfs(s, INF)
                if pushed <= 0:
                    break
                flow += pushed
        return flow

def _Ic_from_JmeV(J_meV):
    e, hbar, kB_eV, eV_to_J = _constants()
    J_eV = J_meV * 1e-3
    J_J = J_eV * eV_to_J
    return (2*e/hbar) * J_J  # Amps

def _maxflow_capacity(Jx_meV):
    N, edges, S, T = _build_lattice()
    Jy_meV = 0.85 * Jx_meV
    Jp_meV = 0.06 * Jx_meV

    # Build Dinic graph with super source/sink
    SRC = N
    SNK = N+1
    D = _Dinic(N+2)

    # infinite-ish capacities for electrode hookups
    INF = 1e50
    for u in S:
        D.add_edge(SRC, u, INF)
    for u in T:
        D.add_edge(u, SNK, INF)

    # add undirected edges as two directed edges with same cap
    for u,v,kind in edges:
        if kind == "x":
            cap = _Ic_from_JmeV(Jx_meV)
        elif kind == "y":
            cap = _Ic_from_JmeV(Jy_meV)
        else:
            cap = _Ic_from_JmeV(Jp_meV)
        D.add_edge(u, v, cap)
        D.add_edge(v, u, cap)

    return D.max_flow(SRC, SNK)

def _lambda2_weighted():
    # Dimensionless weights: wx=1, wy=0.85, wz=0.06
    N, edges, _, _ = _build_lattice()
    wx, wy, wz = 1.0, 0.85, 0.06
    L = np.zeros((N,N), dtype=float)
    for u,v,kind in edges:
        w = wx if kind=="x" else (wy if kind=="y" else wz)
        L[u,u] += w
        L[v,v] += w
        L[u,v] -= w
        L[v,u] -= w
    evals = np.linalg.eigvalsh(L)
    evals.sort()
    return float(evals[1])  # second-smallest

def _coherence_ok(method, Jx_meV, T_star=300.0):
    _, _, kB_eV, _ = _constants()
    thresh = (2.0/math.pi) * kB_eV * T_star  # in eV

    if method in ("BKT_MAXFLOW","BKT_PATH"):
        Jx_eV = Jx_meV * 1e-3
        Jy_eV = 0.85 * Jx_eV
        rho = math.sqrt(Jx_eV * Jy_eV)
        return rho > thresh

    if method == "SPECTRAL_MAXFLOW":
        lam2 = _lambda2_weighted()
        Jx_eV = Jx_meV * 1e-3
        rho_eff = (lam2/4.0) * Jx_eV
        return rho_eff > thresh

    raise ValueError("Unknown method")

def _transport_ok(method, Jx_meV, I_star=5e-3):
    if method in ("BKT_MAXFLOW","SPECTRAL_MAXFLOW"):
        mf = _maxflow_capacity(Jx_meV)
        return mf >= I_star

    if method == "BKT_PATH":
        # 8 edge-disjoint straight paths from x=0->1->2 (fixed y,z)
        Ic_x = _Ic_from_JmeV(Jx_meV)
        return 8.0 * Ic_x >= I_star

    raise ValueError("Unknown method")

def _check(Jx_meV, method):
    return _coherence_ok(method, Jx_meV) and _transport_ok(method, Jx_meV)

def verify_output(out):
    # Basic parsing
    if not isinstance(out, dict):
        raise TypeError("answer() must return a dict with keys 'method' and 'Jx_min_meV'.")

    if "method" not in out or "Jx_min_meV" not in out:
        raise KeyError("Missing required keys. Need 'method' and 'Jx_min_meV'.")

    method = out["method"]
    if method not in ("BKT_MAXFLOW","BKT_PATH","SPECTRAL_MAXFLOW"):
        raise ValueError("method must be one of: BKT_MAXFLOW, BKT_PATH, SPECTRAL_MAXFLOW.")

    Jx = float(out["Jx_min_meV"])
    if not math.isfinite(Jx) or Jx <= 0:
        raise ValueError("Jx_min_meV must be a positive finite number.")
    if not _grid_ok(Jx, 0.1):
        raise ValueError("Jx_min_meV must lie on the 0.1 meV grid (e.g., 12.3).")

    # Correctness
    if not _check(Jx, method):
        raise AssertionError(f"Returned Jx={Jx} meV does NOT satisfy the chosen certificate method {method}.")

    # Grid-minimality
    delta = 0.1
    Jx_prev = Jx - delta
    if Jx_prev > 0 and _check(Jx_prev, method):
        raise AssertionError(
            f"Returned Jx={Jx} meV is not minimal on the 0.1 meV grid for method {method}: "
            f"Jx-delta={Jx_prev} meV also passes."
        )

    return True

def verify_in_notebook():
    out = answer()
    try:
        verify_output(out)
        print("PASS")
        print(f"method={out['method']}, Jx_min_meV={out['Jx_min_meV']}")
    except Exception as e:
        print("FAIL")
        print(str(e))

verifier_source = r'''import importlib.util
import math
import numpy as np
import sys
import traceback

def _constants():
    e = 1.602176634e-19
    hbar = 1.054571817e-34
    kB_eV = 8.617333262145e-5
    eV_to_J = 1.602176634e-19
    return e, hbar, kB_eV, eV_to_J

def _grid_ok(x, delta=0.1, tol=1e-9):
    return abs(x/delta - round(x/delta)) <= tol

def _build_lattice():
    Lx, Ly, Lz = 5, 4, 2
    def idx(x,y,z):
        return (z*Ly + y)*Lx + x
    N = Lx*Ly*Lz
    edges = []
    for z in range(Lz):
        for y in range(Ly):
            for x in range(Lx):
                u = idx(x,y,z)
                v = idx((x+1)%Lx, y, z)
                edges.append((u,v,"x"))
                v = idx(x, (y+1)%Ly, z)
                edges.append((u,v,"y"))
                if z+1 < Lz:
                    v = idx(x,y,z+1)
                    edges.append((u,v,"z"))
    S = [idx(0,y,z) for z in range(Lz) for y in range(Ly)]
    T = [idx(2,y,z) for z in range(Lz) for y in range(Ly)]
    return N, edges, S, T

class _Dinic:
    def __init__(self, n):
        self.n = n
        self.adj = [[] for _ in range(n)]
    def add_edge(self, u, v, cap):
        self.adj[u].append([v, cap, len(self.adj[v])])
        self.adj[v].append([u, 0.0, len(self.adj[u])-1])
    def max_flow(self, s, t):
        flow = 0.0
        INF = 1e100
        while True:
            level = [-1]*self.n
            q = [s]
            level[s] = 0
            for u in q:
                for v, cap, rev in self.adj[u]:
                    if cap > 1e-15 and level[v] < 0:
                        level[v] = level[u] + 1
                        q.append(v)
            if level[t] < 0:
                break
            it = [0]*self.n
            def dfs(u, f):
                if u == t:
                    return f
                for i in range(it[u], len(self.adj[u])):
                    it[u] = i
                    v, cap, rev = self.adj[u][i]
                    if cap > 1e-15 and level[v] == level[u] + 1:
                        ret = dfs(v, min(f, cap))
                        if ret > 0:
                            self.adj[u][i][1] -= ret
                            self.adj[v][rev][1] += ret
                            return ret
                return 0.0
            while True:
                pushed = dfs(s, INF)
                if pushed <= 0:
                    break
                flow += pushed
        return flow

def _Ic_from_JmeV(J_meV):
    e, hbar, kB_eV, eV_to_J = _constants()
    J_eV = J_meV * 1e-3
    J_J = J_eV * eV_to_J
    return (2*e/hbar) * J_J

def _maxflow_capacity(Jx_meV):
    N, edges, S, T = _build_lattice()
    Jy_meV = 0.85 * Jx_meV
    Jp_meV = 0.06 * Jx_meV
    SRC = N
    SNK = N+1
    D = _Dinic(N+2)
    INF = 1e50
    for u in S:
        D.add_edge(SRC, u, INF)
    for u in T:
        D.add_edge(u, SNK, INF)
    for u,v,kind in edges:
        if kind == "x":
            cap = _Ic_from_JmeV(Jx_meV)
        elif kind == "y":
            cap = _Ic_from_JmeV(Jy_meV)
        else:
            cap = _Ic_from_JmeV(Jp_meV)
        D.add_edge(u, v, cap)
        D.add_edge(v, u, cap)
    return D.max_flow(SRC, SNK)

def _lambda2_weighted():
    N, edges, _, _ = _build_lattice()
    wx, wy, wz = 1.0, 0.85, 0.06
    L = np.zeros((N,N), dtype=float)
    for u,v,kind in edges:
        w = wx if kind=="x" else (wy if kind=="y" else wz)
        L[u,u] += w
        L[v,v] += w
        L[u,v] -= w
        L[v,u] -= w
    evals = np.linalg.eigvalsh(L)
    evals.sort()
    return float(evals[1])

def _coherence_ok(method, Jx_meV, T_star=300.0):
    _, _, kB_eV, _ = _constants()
    thresh = (2.0/math.pi) * kB_eV * T_star
    if method in ("BKT_MAXFLOW","BKT_PATH"):
        Jx_eV = Jx_meV * 1e-3
        Jy_eV = 0.85 * Jx_eV
        rho = math.sqrt(Jx_eV * Jy_eV)
        return rho > thresh
    if method == "SPECTRAL_MAXFLOW":
        lam2 = _lambda2_weighted()
        Jx_eV = Jx_meV * 1e-3
        rho_eff = (lam2/4.0) * Jx_eV
        return rho_eff > thresh
    raise ValueError("Unknown method")

def _transport_ok(method, Jx_meV, I_star=5e-3):
    if method in ("BKT_MAXFLOW","SPECTRAL_MAXFLOW"):
        mf = _maxflow_capacity(Jx_meV)
        return mf >= I_star
    if method == "BKT_PATH":
        Ic_x = _Ic_from_JmeV(Jx_meV)
        return 8.0 * Ic_x >= I_star
    raise ValueError("Unknown method")

def _check(Jx_meV, method):
    return _coherence_ok(method, Jx_meV) and _transport_ok(method, Jx_meV)

def _load_submission(path: str):
    spec = importlib.util.spec_from_file_location("submission", path)
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Could not load submission from: {path}")
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

def verify(path: str) -> int:
    try:
        sub = _load_submission(path)
    except Exception as e:
        print("FAIL: could not import submission.")
        print(f"Error: {e}")
        print(traceback.format_exc())
        return 1
    if not hasattr(sub, "answer") or not callable(sub.answer):
        print("FAIL: submission must define a callable function: answer()")
        return 1
    try:
        out = sub.answer()
    except Exception as e:
        print("FAIL: answer() raised an exception.")
        print(f"Error: {e}")
        print(traceback.format_exc())
        return 1
    try:
        if not isinstance(out, dict):
            raise TypeError("answer() must return a dict.")
        method = out.get("method", None)
        Jx = out.get("Jx_min_meV", None)
        if method not in ("BKT_MAXFLOW","BKT_PATH","SPECTRAL_MAXFLOW"):
            raise ValueError("method must be one of: BKT_MAXFLOW, BKT_PATH, SPECTRAL_MAXFLOW.")
        Jx = float(Jx)
        if not math.isfinite(Jx) or Jx <= 0:
            raise ValueError("Jx_min_meV must be positive and finite.")
        if not _grid_ok(Jx, 0.1):
            raise ValueError("Jx_min_meV must lie on the 0.1 meV grid.")
        if not _check(Jx, method):
            raise AssertionError(f"Returned Jx={Jx} meV does NOT satisfy method {method}.")
        delta = 0.1
        if Jx - delta > 0 and _check(Jx - delta, method):
            raise AssertionError(f"Returned Jx={Jx} meV is not minimal on-grid for method {method}.")
    except Exception as e:
        print("FAIL")
        print(str(e))
        return 1
    print("PASS")
    print(f"method={method}, Jx_min_meV={Jx}")
    return 0

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python verifier.py /path/to/submission.py")
        sys.exit(2)
    sys.exit(verify(sys.argv[1]))
'''
with open("verifier.py", "w", encoding="utf-8") as f:
    f.write(verifier_source)

print("Wrote verifier.py")
print("Tip: in this notebook, run verify_in_notebook() after you fill in answer().")
i2Evscode-notebook-cell:/Users/richa/Noether/Emily_envs/Challenge2.ipynb